# Redes neuronales artificiales

Durante las últimas clases haremos una exploración sobre varias técnicas de aprendizaje profundo (deep learning) comenzando con las redes neuronales artificiales (ANN), en particular las redes neuronales Feedforward. 

Usaremos el framework Keras, que es una API de alto nivel además de Tensorflow. Keras se está volviendo muy popular recientemente debido a su simplicidad. Es muy fácil crear modelos complejos e iterar rápidamente. 

## Introducción

Las redes neuronales artificiales (ANN) son redes neuronales conformadas por varias capas de neuronas totalmente conectadas:

![img1](https://raw.githubusercontent.com/Izainea/visualizacion/master/img/ANN1.png)


Consisten en una capa de entrada, varias capas ocultas y una capa de salida. Cada nodo de una capa está conectado a todos los demás nodos de la siguiente capa. Representa lo que estudiamos hace un momento, el perceptrón:

![img2](https://raw.githubusercontent.com/Izainea/visualizacion/master/img/ANN2.png)

Después de que cada perceptrón aplica los pesos obtenidos y se tiene la salida después de aplicar la función de activación entonces cada salida se convierte en la entrada para la siguiente capa. Los cálculos fluyen el diagrama de izquierda a derecha y la salida final se calcula realizando este procedimiento para todos los nodos. 

El objetivo de esta red neuronal profunda es aprender los pesos asociados a cada flecha de la primera gráfica, en otras palabras, consiste en la estimación de las siguientes matrices:

![img3](https://raw.githubusercontent.com/Izainea/visualizacion/master/img/ANN3.png)

En ese sentido, aprovechando el gráfico anterior, entendemos que la salida, para la red representada en esa figura, se calcula de la siguiente forma:

$$y=f(f(f(x\cdot W_1)\cdot W_2)\cdot W_3)$$

En este caso, todos los perceptrones tienen la misma función de activación $f$. El sesgo no esta incluido en la fórmula anterior, pero podemos ignorarlo mientras concebimos la intuición detrás de estas redes neuronales.

## Backpropagation 
Hasta ahora hemos descrito el cálculo de la salida a partir de unos pesos estimados pero ¿Cómo estimamos estos pesos?, en otras palabras, ¿Cómo entrenamos esta red? 

El siguiente algoritmo describe este proceso:

* Inicialice aleatoriamente los pesos de todos los nodos. 

* La salida final es el valor del último nodo. Compare la salida final con el objetivo real en los datos de entrenamiento y mida el error usando una función de pérdida (Loss function).

* Realice un pase hacia atrás de derecha a izquierda y propague el error a cada nodo individual utilizando la propagación hacia atrás. * Calcule la contribución de cada peso al error y ajuste los pesos en consecuencia utilizando el descenso de gradiente. Propague los gradientes de error a partir de la última capa.

La propagación hacia atrás con descenso de gradiente es el concepto fundamental detrás del entrenamiento de la red, en esencia buscamos minimizar el error y en ese sentido buscamos un valor mínimo de la función de pérdida. Los detalles matemáticos los omitiremos de esta explicación pero se pueden encontrar en este video:
[![imgvid](https://img.youtube.com/vi/9OzLcgy1bjs/0.jpg)](https://www.youtube.com/watch?v=9OzLcgy1bjs)



## ¿Por qué funcionan las redes?

La esencia de las redes neuronales, por lo menos en esta versión inicial, consiste en la posibilidad de proyectar-transformar los registros de entrada en un espacio con mayor dimensión, con eso el proceso de clasificación se hace más sencillo, el siguiente gráfico ilustra esta situación:

![img4](https://raw.githubusercontent.com/Izainea/visualizacion/master/img/ANN4.png)

La proyección a otra dimensión permitió que hicieramos una separación como la siguiente:

![img5](https://raw.githubusercontent.com/Izainea/visualizacion/master/img/ANN5.png)

En resumen, las ANN son modelos de aprendizaje profundo muy flexibles pero potentes, permiten estimar aproximaciones a cualquier función compleja. Su aumento de popularidad se ha debido a tres razones: trucos inteligentes que hicieron posible el entrenamiento de estos modelos, un gran aumento en la potencia computacional, especialmente GPU y entrenamiento distribuido, además de una gran cantidad de datos de entrenamiento.





## Implementación en Python

Vamos ahora a usar el framework `keras` para hacer algunas  clasificaciones. Iniciemos con lo que necesitamos para poder iniciar el desarrollo de los modelos:

In [ ]:
!pip install tensorflow

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

import numpy as np
import seaborn as sns
import warnings

#warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)


from datetime import datetime
from matplotlib.colors import ListedColormap
from sklearn.datasets import make_classification, make_moons, make_circles
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold
import keras.backend as K
from keras.wrappers.scikit_learn import KerasClassifier

`make_classification` es una función que nos permite simular un data set para clasificar:

In [ ]:
X, y = make_classification(n_samples=1000, n_features=2, n_redundant=0, 
                           n_informative=2, random_state=2020, n_clusters_per_class=1)
sns.scatterplot(x=X.T[0],y=X.T[1],hue=y)

Antes de usar `keras` veamos un clasificador lineal, la regresión logística, compararemos después como clasifica una red neuronal:

In [ ]:
lr = LogisticRegression()
lr.fit(X, y)
y_pred=lr.predict(X)
sns.heatmap(confusion_matrix(y,y_pred),annot=True,fmt='.0f')

In [ ]:
sns.scatterplot(x=X.T[0],y=X.T[1],hue=y,style=y_pred)
limits = np.array([-4, 4])
boundary = -(lr.coef_[0][0] * limits + lr.intercept_[0]) / lr.coef_[0][1]
plt.plot(limits, boundary, "g-", linewidth=2)

Ahora veamos que ocurre con `keras`, despleguemos el modelo de clasificación usando la sintaxis de keras, la documentación se puede encontrar aquí:
[Documentación Keras](https://keras.io/)

In [ ]:
model = Sequential()
model.add(Dense(units=1, input_shape=(2,), activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(x=X, y=y, verbose=0, epochs=100)


In [ ]:
model.summary()

In [ ]:
y_pred=model.predict_classes(X)
sns.heatmap(confusion_matrix(y,y_pred),annot=True,fmt='.0f')

In [ ]:
history.history

In [ ]:
def plot_loss_accuracy(history):
    historydf = pd.DataFrame(history.history, index=history.epoch)
    plt.figure(figsize=(8, 6))
    historydf.plot(ylim=(0, max(1, historydf.values.max())))
    loss = history.history['loss'][-1]
    acc = history.history['accuracy'][-1]
    plt.title('Loss: %.3f, Accuracy: %.3f' % (loss, acc))
plot_loss_accuracy(history)

In [ ]:
model = Sequential()
model.add(Dense(units=10, input_shape=(2,), activation='sigmoid'))
#model.add(Dense(units=4, input_shape=(4,), activation='sigmoid'))
model.add(Dense(units=1, input_shape=(10,), activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(x=X, y=y, verbose=0, epochs=500)


In [ ]:
model.summary()

In [ ]:
y_pred=model.predict_classes(X)
sns.heatmap(confusion_matrix(y,y_pred),annot=True,fmt='.0f')

In [ ]:
plot_loss_accuracy(history)

In [ ]:
sns.scatterplot(x=X.T[0],y=X.T[1],hue=y,style=y_pred.T[0])